In [1]:
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

     |████████████████████████████████| 719 kB 31.6 MB/s 
     |████████████████████████████████| 4.7 MB 65.6 MB/s 
     |████████████████████████████████| 365 kB 89.2 MB/s 
     |████████████████████████████████| 1.3 MB 54.8 MB/s 
     |████████████████████████████████| 120 kB 101.5 MB/s 
     |████████████████████████████████| 115 kB 101.8 MB/s 
     |████████████████████████████████| 212 kB 75.4 MB/s 
     |████████████████████████████████| 127 kB 71.3 MB/s 
     |████████████████████████████████| 6.6 MB 71.0 MB/s 
Mounted at /content/gdrive


In [2]:
from fastai.vision.all import *
from fastbook import *


In [3]:
path = untar_data(URLs.MNIST)

In [4]:
Path.BASE_PATH = path

In [5]:

dls=DataBlock((ImageBlock(cls=PILImageBW),CategoryBlock),
                   get_items=get_image_files,
                   get_y = parent_label,
                   splitter = GrandparentSplitter('training','testing'),
                   batch_tfms=Normalize()).dataloaders(path,bs=64)



In [6]:
def fit(epochs=1, lr=0.06):
    learn = Learner(dls, simple_cnn(), loss_func=F.cross_entropy,
                    metrics=accuracy, cbs=ActivationStats(with_hist=True))
    learn.fit_one_cycle(epochs, lr)
    return learn

In [7]:
def conv(ni, nf, ks=3, act=True):
    layers = [nn.Conv2d(ni, nf, stride=2, kernel_size=ks, padding=ks//2)]
    if act: layers.append(nn.ReLU())
    layers.append(nn.BatchNorm2d(nf))
    return nn.Sequential(*layers)

In [8]:
def simple_cnn():
    return sequential(
        conv(1 ,8, ks=5),        #14x14
        conv(8 ,16),             #7x7
        conv(16,32),             #4x4
        conv(32,64),             #2x2
        conv(64,10, act=False),  #1x1
        Flatten(),
    )

In [9]:
learn = fit(5, lr=0.1)

/usr/local/lib/python3.7/dist-packages/fastai/callback/core.py:69: UserWarning: You are shadowing an attribute (modules) that exists in the learner. Use `self.learn.modules` to avoid this
  warn(f"You are shadowing an attribute ({name}) that exists in the learner. Use `self.learn.{name}` to avoid this")


epoch,train_loss,valid_loss,accuracy,time
0,0.142462,0.097402,0.968700,01:00
1,0.086911,0.090723,0.971900,01:00
2,0.076816,0.069186,0.979200,00:59
3,0.034913,0.031329,0.991000,01:00
4,0.019268,0.023806,0.993000,01:00


In [10]:
learn.summary()

Sequential (Input shape: 64 x 1 x 28 x 28)
Layer (type)         Output Shape         Param #    Trainable 
                     64 x 8 x 14 x 14    
Conv2d                                    208        True      
ReLU                                                           
BatchNorm2d                               16         True      
____________________________________________________________________________
                     64 x 16 x 7 x 7     
Conv2d                                    1168       True      
ReLU                                                           
BatchNorm2d                               32         True      
____________________________________________________________________________
                     64 x 32 x 4 x 4     
Conv2d                                    4640       True      
ReLU                                                           
BatchNorm2d                               64         True      
_____________________________________

In [11]:
learn.export()